# Hands-on IFC data exploration
24.06.2025 Zurich

Following are a series of activities that are a part of BIM and LLMs in LCA workshop, SBE25. The aim is exploring the underlying data structure within an IFC model and learning ways to extract bill of materials from it.

## First of all: IFC file - what is actually inside?

## IFC model import

In [ ]:
!pip install ifcopenshell
!pip install pandas
import ifcopenshell
import ifcopenshell.util.element

In [ ]:
model = ifcopenshell.open("./sbe25/data/IFC_models/1072_HiLo_Model-Architecture-IFC4.ifc")

In [ ]:
project = model.by_type("IfcProject")[0]
print(project)

This is a string representation of an object instance, of the `IfcProject` entity.

## Accessing the model contents of an IFC file
### `by_type()` function


### Finding all the building elements:

In [ ]:
buildingelements = model.by_type("IfcBuildingElement")

print(buildingelements)

### Let's make it a bit easier to read:

In [ ]:
for buildingelement in buildingelements:
    print(buildingelement)

### What other ways are there to find building elements? `by_guid()`, `by_id()` functions

In [ ]:
object2 = model.by_id('write the ID here')  # Replace with any ID from the model

print(object2)

In [ ]:
object = model.by_guid("write the GUID here") # Replace with any GUID from the model

print(object)

### What about an IFC viewer? - Can we find these objects there?

In [ ]:
from IPython.display import IFrame
IFrame(src='https://app.speckle.systems/projects/8d7bac3e0a/models/ce1e2a8ad7', width=1200, height=600)

### How many are there in total? `len()` function

In [ ]:
len(buildingelements)

### Let's pick one object and have a closer look:

In [ ]:
buildingelement = buildingelements[80]

buildingelement.get_info()

`model.get_inverse(b)` returns a list of all IFC entities that refer to the entity `b` through inverse relationships, as defined by the IFC schema. This includes entities like `IfcRelContainedInSpatialStructure`, `IfcRelAssociates`, etc., which point to `b` in their forward relations.


In [ ]:
model.get_inverse(buildingelement)

### Where is this element in the model hierarchy?

`model.traverse(buildingelement, max_levels=1)` returns a dictionary of the direct attributes of IFC entity `buildingelement`. It expands only one level deep into nested IFC entities, making it useful for shallow inspection of structure and references.


In [ ]:
model.traverse(buildingelement, max_levels=1)

### Now let's try to find all the wall instances in the project:

In [ ]:
# Find all the IfcWall instances (use walls as the variable name)

In [ ]:
# Create a loop to print all walls

In [ ]:
# How many walls are there?

In [ ]:
# Pick one wall and print its information

### Can you filter IfcWalls in the 3D IFC-viewer?

In [ ]:
from IPython.display import IFrame
IFrame(src='https://app.speckle.systems/projects/8d7bac3e0a/models/ce1e2a8ad7', width=1200, height=600)

## Properties

### How can we see the properties of building elements? Here is an example for one.

In [ ]:
# Pick the first one from the list of building elements
buildingelem = buildingelements[0]

print(buildingelem)

In [ ]:
ifcopenshell.util.element.get_psets(buildingelem)

### Now, let's list the properties of a wall together.

In [ ]:
# Pick a wall from the list of walls
wall = walls[50]

In [ ]:
# List the property sets of the wall
ifcopenshell.util.element.get_psets(wall)

### Is this wall interior or exterior?

In [ ]:
psets_wall = ifcopenshell.util.element.get_psets(wall)

psets_wall["Construction"].get("Function")

### Can we see this information in the 3D IFC-viewer?

In [ ]:
from IPython.display import IFrame
IFrame(src='https://app.speckle.systems/projects/8d7bac3e0a/models/ce1e2a8ad7', width=1200, height=600)

### Is it a structural wall? Try to find it out yourself.

In [ ]:
# Pick a wall from the list



# List the property sets of the wall



# Try to find out if it is a load-bearing wall


### Can we find all the exterior walls and them to a list?

In [ ]:
exterior_walls = []

for wall in walls:
    psets = ifcopenshell.util.element.get_psets(wall)

    try:
        if psets["Construction"].get("Function") == 'Exterior':
            exterior_walls.append(wall)
            print(wall)
    except KeyError:
        pass

### Now, can you find all the load-bearing/structural walls and them to a list?

In [ ]:
# Find all the IfcWall instances and check if they are load-bearing



## Quantities

### Can we measure the quantities of model elements? (such as dimensions, area, volume and so on)

In [ ]:
# Pick the first wall (or any element you want)
wall = model.by_type("IfcWall")[80]

#list the property sets of the wall
ifcopenshell.util.element.get_psets(wall)


### Let's look at the quantities more closely. Can you print out the width, height or area of a wall?

In [ ]:
# Try to find the quantity values



### Another method to print out all the properties within a property set:

In [ ]:
# Get the BaseQuantities property set
base_quantities = psets.get("BaseQuantities", {})

### Do all walls have base quantities?

In [ ]:
for wall in walls:

    psets = ifcopenshell.util.element.get_psets(wall)

    base_quantities = psets.get("BaseQuantities", {})

    print(base_quantities)

### Let's find all the walls that have height as a base quantity.

In [ ]:
for wall in walls:
    psets = ifcopenshell.util.element.get_psets(wall)

    base_quantities = psets.get("BaseQuantities", {})

    if 'Height' in base_quantities:
        print(f"Wall {wall.Name} has a height of {base_quantities['Height']}")


### Let's find all the walls higher than 2 metres.

In [ ]:
for wall in walls:
    psets = ifcopenshell.util.element.get_psets(wall)

    base_quantities = psets.get("BaseQuantities", {})

    if 'Height' in base_quantities:
        height = base_quantities['Height']

        if height > 2000:
            print(f"Wall {wall.Name} has a height of {height}, which is greater than 2000 mm.")


### Can you check if the selected wall is thicker than 15 mm?

In [ ]:
# Check whether the wall is thicker than 15 mm

### Can you find all the walls thicker than 15 mm?

In [ ]:
# Find all the walls thicker than 15 mm - don't forget to check if the base quantity exists

### Let's check it also in the 3D IFC-viewer.

In [ ]:
from IPython.display import IFrame
IFrame(src='https://app.speckle.systems/projects/8d7bac3e0a/models/ce1e2a8ad7', width=1200, height=600)

## Materials

### Can we see the layers of materials within a composite structure?

In [ ]:
def print_material_layers(el):
    for assoc in getattr(el, "HasAssociations", []):
        if assoc.is_a("IfcRelAssociatesMaterial"):
            mat = assoc.RelatingMaterial
            if mat.is_a("IfcMaterialLayerSetUsage"):
                for layer in mat.ForLayerSet.MaterialLayers:
                    print(f"Material: {layer.Material.Name}, Thickness: {layer.LayerThickness} mm")


In [ ]:
# Try on a wall
wall = model.by_type("IfcWall")[200]

print(f"{getattr(wall, 'Name', '')}")

print_material_layers(wall)

## Exporting Data to an Excel List

### How can we export the IFC model data when needed?

In [ ]:
import pandas as pd
from collections import defaultdict

In [ ]:
# Element types to include
element_types = ["IfcWall", "IfcSlab", "IfcBeam", "IfcColumn", "IfcDoor", "IfcWindow"]

# Group elements by Name
grouped_elements = defaultdict(list)

for element_type in element_types:
    for el in model.by_type(element_type):
        name = getattr(el, "ObjectType", "").strip() or "Unnamed"
        grouped_elements[name].append(el)
        print(name)
        print(el)


# Summarize each group
summary = []

for name, group in grouped_elements.items():
    representative = group[0]
    ifc_type = representative.is_a()
    object_type = getattr(representative, "ObjectType", "") or ""

    material_layers = set()

    for el in group:
        for assoc in getattr(el, "HasAssociations", []):
            if assoc.is_a("IfcRelAssociatesMaterial"):
                mat = assoc.RelatingMaterial

                if mat.is_a("IfcMaterial"):
                    material_layers.add(mat.Name)

                elif mat.is_a("IfcMaterialLayerSetUsage"):
                    for layer in mat.ForLayerSet.MaterialLayers:
                        if layer.Material:
                            material_layers.add(layer.Material.Name)

                elif mat.is_a("IfcMaterialProfileSetUsage"):
                    for profile in mat.ForProfileSet.MaterialProfiles:
                        if profile.Material:
                            material_layers.add(profile.Material.Name)

                elif mat.is_a("IfcMaterialList"):
                    for m in mat.Materials:
                        if m:
                            material_layers.add(m.Name)

    material_desc = " | ".join(sorted(material_layers)) if material_layers else "No material"

    summary.append({
        "Name": name,
        "Count": len(group),
        "IFC Type": ifc_type,
        "Material Layers": material_desc
    })

# Export to DataFrame
df = pd.DataFrame(summary)

# Save to Excel
df.to_excel("grouped_building_elements_summary.xlsx", index=False)

# Display
df.head()